In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
import optuna.integration.lightgbm as lgb_o
from itertools import combinations, permutations
import matplotlib.pyplot as plt
from selenium import webdriver
from urllib.parse import urljoin
from chardet import detect

In [2]:
class RaceResultsSP:
    def __init__(self):
        self.user_id = 'ken.osechi@gmail.com'
        self.password = 'keson0213'
        self.login_info = {
            "login_id": self.user_id,
            "pswd": self.password,
        }
        self.session = requests.session()
        self.url_login = "https://regist.netkeiba.com/account/?pid=login&action=auth"
        self.ses = self.session.post(self.url_login, data=self.login_info)
        self.cookie = self.ses.cookies


    def scrape(self, race_id_list):
        #race_idをkeyにしてDataFrame型を格納
        race_results = {}
        odds_results = {}
        lap_results = {}
        for race_id in tqdm(race_id_list):
            time.sleep(1)
            try:
                url = "https://db.sp.netkeiba.com/race/" + race_id  
                html = self.session.get(url, cookies=self.cookie)
                html.encoding = "EUC-JP"
                soup = BeautifulSoup(html.content, "html.parser")
                race_data = soup.find('div', class_='RaceData').find_all('span')
                race_data = ''.join(tag.get_text() for tag in race_data)
    
                #htmlからBeautifulSoupでデータを抽出
                df = pd.read_html(html.text)[0]
                df = df.rename(columns=lambda x: x.replace(' ', '')) #カラム名の空白削除
                #horse_id
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"class": "table_slide_body ResultsByRaceDetail"}).find_all('tbody')[0].find_all(
                    "a", attrs={"href": re.compile(r'horse/(\d+)/')})
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                horse_id_list
                df['horse_id'] = horse_id_list
                #indexをrace_idにする
                df.index = [race_id]*len(df)    
                race_results[race_id] = df

                #オッズの取得
                # odds = pd.read_html(html.text)[1]
                # type = odds.iloc[:, 0]
                # win = odds.iloc[:, 2]
                # odds_df = pd.concat([type, win], axis=1).set_index(0).T
                # odds_df.index = [race_id]
                # odds_results[race_id] = odds_df

                #laptimeの取得
                len_ls = [i for i in range(100, 2600, 100)] + [i+50 for i in range(100, 1200, 100)] + [i for i in range(2600, 3601, 200)]
                len_ls = sorted(len_ls)
                len_ls = [f'{i}m' for i in len_ls]
                lap_df = pd.DataFrame(columns=len_ls)
                if '障' in race_data:
                    lap_df.loc[0] = np.nan
                else:
                    lap_time_read = pd.read_html(html.text)[5]
                    lap_time_read = lap_time_read.T.reset_index().T.reset_index(drop=True)

                    for i in range(lap_time_read.shape[0]):
                        if i %2 == 1:
                            pass
                        else:
                            for j in range(lap_time_read.shape[1]):
                                if pd.isna(lap_time_read.iloc[i, j]):
                                    break            
                                length = str(lap_time_read.iloc[i, j])
                                lap_df[length] = [float(lap_time_read.iloc[i+1, j].split()[1])]
                lap_df.index = [race_id]
                lap_results[race_id] = lap_df
              
            #存在しないrace_idを飛ばす
            except IndexError:
                continue
            except AttributeError: #存在しないrace_idでAttributeErrorになるページもあるので追加
                continue
            #wifiの接続が切れた時などでも途中までのデータを返せるようにする
            except Exception as e:
                print(e)
                break
            #Jupyterで停止ボタンを押した時の対処
            except:
                break

        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])
        # odds_results_df = pd.concat([odds_results[key] for key in odds_results], ignore_index=True)
        lap_results_df = pd.concat([lap_results[key] for key in lap_results])

        return race_results_df, lap_results_df



In [3]:
class Return:
    @staticmethod
    def scrape(race_id_list):
        """
        払い戻し表データをスクレイピングする関数

        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト

        Returns:
        ----------
        return_tables_df : pandas.DataFrame
            全払い戻し表データをまとめてDataFrame型にしたもの
        """

        return_tables = {}
        for race_id in tqdm(race_id_list):
            time.sleep(1)
            try:
                url = "https://db.netkeiba.com/race/" + race_id

                #普通にスクレイピングすると複勝やワイドなどが区切られないで繋がってしまう。
                #そのため、改行コードを文字列brに変換して後でsplitする
                f = urlopen(url)
                html = f.read()
                html = html.replace(b'<br />', b'br')
                dfs = pd.read_html(html)

                #dfsの1番目に単勝〜馬連、2番目にワイド〜三連単がある
                df = pd.concat([dfs[1], dfs[2]])

                df.index = [race_id] * len(df)
                return_tables[race_id] = df
            except IndexError:
                continue
            except AttributeError: #存在しないrace_idでAttributeErrorになるページもあるので追加
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #pd.DataFrame型にして一つのデータにまとめる
        return_tables_df = pd.concat([return_tables[key] for key in return_tables])
        return return_tables_df
race_id_list = ['201701010101', '201701010102']
a = Return().scrape(race_id_list)
a

  0%|          | 0/2 [00:00<?, ?it/s]

,0,1,2,3
201701010101,単勝,3,300,2
201701010101,複勝,3br5,130br110,2br1
201701010101,馬連,3 - 5,160,1
201701010101,ワイド,3 - 5br3 - 7br5 - 7,110br160br130,1br3br2
201701010101,馬単,3 → 5,360,2
201701010101,三連複,3 - 5 - 7,160,1
201701010101,三連単,3 → 5 → 7,810,3
201701010102,単勝,10,300,1
201701010102,複勝,10br12br5,120br190br150,1br4br3
201701010102,枠連,7 - 8,480,2


In [13]:
import os
race = pd.read_pickle('race.pickle')
race_id_list_all = race.index.unique()
current_dir = os.getcwd()
for i in range(18000, len(race_id_list_all), 1000):
    race_id_list = race_id_list_all[i:i+1000]   
    race_results_df, lap_results_df = RaceResultsSP().scrape(race_id_list)
    race_dir = os.path.join(current_dir, 'race_res', f'race_{i}.pickle')
    lap_dir = os.path.join(current_dir, 'lap', f'lap_{i}.pickle')
    # odds_dir = os.path.join(current_dir, 'odds', f'odds_{i}.pickle')
    race_results_df.to_pickle(race_dir)
    lap_results_df.to_pickle(lap_dir)
    # odds_results_df.to_pickle(odds_dir)



  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/553 [00:00<?, ?it/s]

In [12]:
import os
current_dir = os.getcwd()
for i in range(0, 19000, 1000):
    race_dir = os.path.join(current_dir, 'race_res', f'race_{i}.pickle')
    lap_dir = os.path.join(current_dir, 'lap', f'lap_{i}.pickle')
    p = pd.read_pickle(race_dir)
    l = pd.read_pickle(lap_dir)
    print(len(l))



978
1000
1000
978
1000
1000
993
985
1000
1000
978
1000
1000
976
1000
1000
980
1000
399


In [25]:
import pandas as pd
import numpy as np
a = pd.read_pickle('chokyo/chokyo_1000.pickle')


2013103950    南Ｗ
2013103950    美Ｐ
2013103950    美Ｐ
2013103950    美Ｐ
2013103950    美Ｐ
              ..
2014101414    函Ｗ
2014101414    函Ｗ
2014101414    函Ｗ
2014101414    美坂
2014101414    美坂
Name: place, Length: 62023, dtype: object

In [27]:
LAP = pd.DataFrame()
RACE = pd.DataFrame()

for i in range(0, 20000, 1000):
    lap = pd.read_pickle(f'lap/lap_{i}.pickle')
    race = pd.read_pickle(f'race_res/race_{i}.pickle')
    LAP = pd.concat([LAP, lap])
    RACE = pd.concat([RACE, race])
RACE.to_pickle('race.pickle')
LAP.to_pickle('lap.pickle')

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,タイム指数,...,単勝,人気,馬体重,調教タイム,厩舎コメント,備考,調教師,馬主,賞金（万円）,horse_id
201701010101,1,3,3,リープフラウミルヒ,牝2,54.0,丹内祐次,1:49.4,NaN,79.0,...,3.0,2.0,406(-4),NaN,NaN,NaN,[東]相沢郁,サラブレッドクラブ・ラフィアン,500.0,2015100713
201701010101,2,5,5,ディバインブリーズ,牡2,54.0,ルメール,1:49.5,1/2,78.0,...,1.5,1.0,484(-4),NaN,NaN,NaN,[西]須貝尚介,杉山忠国,200.0,2015103211
201701010101,3,7,7,スズカワークシップ,牡2,54.0,福永祐一,1:50.1,3.1/2,72.0,...,6.2,3.0,450(-16),NaN,NaN,NaN,[西]橋田満,永井啓弍,130.0,2015103578
201701010101,4,1,1,ハイタイド,牡2,54.0,蛯名正義,1:50.1,クビ,73.0,...,31.1,5.0,442(-10),NaN,NaN,NaN,[東]栗田徹,カナヤマホールディングス,75.0,2015101520
201701010101,5,2,2,クリノバルテュス,牡2,53.0,木幡初也,1:51.3,7,59.0,...,22.8,4.0,472(-18),NaN,NaN,NaN,[東]伊藤伸一,栗本博晴,50.0,2015101217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202308010812,10,2,2,サクセスエース,牡5,58.0,国分優作,1:56.9,1.1/2,60.0,...,15.8,5.0,484(-4),NaN,NaN,NaN,[東]竹内正洋,ブルースターズファーム,NaN,2018104460
202308010812,11,3,3,テイエムファクター,牡4,58.0,幸英明,1:57.4,3,56.0,...,17.1,6.0,452(-8),NaN,NaN,出遅れ,[西]木原一良,竹園正繼,NaN,2019103984
202308010812,12,7,11,ジュンブルースカイ,牡5,58.0,西村淳也,1:58.2,5,49.0,...,4.8,2.0,456(+8),NaN,NaN,NaN,[西]友道康夫,河合純二,NaN,2018105085
202308010812,13,5,8,キゾク,牡5,58.0,鮫島良太,1:58.4,1,47.0,...,264.1,14.0,494(+2),NaN,NaN,NaN,[西]西村真幸,西村憲人,NaN,2018103993


In [29]:
RACE.columns

Index(['着順', '枠番', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', 'タイム指数', '通過',
       '上り', '単勝', '人気', '馬体重', '調教タイム', '厩舎コメント', '備考', '調教師', '馬主', '賞金（万円）',
       'horse_id'],
      dtype='object')